In [1]:
import json

# Replace 'your_file.json' with the path to your JSON file
file_path = 'embeddings.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# Accessing the first item in the list
first_item = data[0]
label = first_item['label']
embedding = first_item['embedding']

print(f"Label: {label}, Embedding: {embedding}")


Label: Tomato___Late_blight, Embedding: ['0.000834583654', '0.0120863235', '0.0167419109', '0.0165973883', '-0.0165261012', '-0.0280585624', '-0.00215590093', '-0.00546082668', '-0.0291261058', '-0.00558335427', '0.0319736935', '-0.002958728', '0.00160852843', '0.0867289677', '0.00936508924', '-0.0136400415', '0.0385997035', '0.0118101267', '-0.00179401448', '7.53525383e-05', '0.0206145328', '-0.025409827', '-0.0280903466', '0.0224383026', '-0.0448605306', '-0.0458414033', '0.00253705401', '0.0136687392', '-0.0460221544', '-0.0174893923', '-0.0299517289', '-0.0115787359', '-0.00974194054', '-0.00716886111', '-0.0226824824', '0.015426904', '-0.0377103', '-0.00990028679', '0.0166591574', '0.0118665304', '-0.000749780913', '0.00844145', '-0.0226943828', '0.0192710627', '-0.0234087836', '-0.0293418337', '0.0045147161', '-0.000593695964', '0.00613935571', '-0.00998749491', '0.0149501786', '-0.0140852397', '-0.0138488812', '0.0153533872', '-0.0296383835', '0.0202457104', '-0.0435836315', '0.

In [23]:
!pip install milvus pymilvus -q  

ERROR: Invalid requirement: '#'


In [61]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, MilvusClient
import os
from dotenv import load_dotenv
load_dotenv()  # load environment variables from .env
CLUSTER_ENDPOINT= 'https://in03-14b128e9f80ae3d.api.gcp-us-west1.zillizcloud.com'
os.environ["TOKEN"]=os.getenv("TOKEN-TWO") # Set your token
COLLECTION_NAME="plant_diseases" # Set your collection name


connections.connect(
  alias='default', 
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=os.environ["TOKEN"], 
)

client = MilvusClient(uri=CLUSTER_ENDPOINT, token=os.environ["TOKEN"])


In [62]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Create collection which includes the id, filepath of the image, and image embedding
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='label', dtype=DataType.VARCHAR, max_length=200),  
    FieldSchema(name='image_embedding', dtype=DataType.FLOAT_VECTOR, dim=1000)
]

schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [63]:
index_params = {
    "index_type": "AUTOINDEX",
    "metric_type": "COSINE",
    "params": {}
}

collection.create_index(
  field_name="image_embedding", 
  index_params=index_params,
  index_name='image_embedding_index'
)

collection.load()

progress = utility.loading_progress(COLLECTION_NAME)

print(progress)


{'loading_progress': '100%'}


In [57]:
import os
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import torch

# Initialize the feature extractor and model
extractor = AutoFeatureExtractor.from_pretrained("microsoft/resnet-50")
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-50")

def process_image(image_path):
    # Open the image
    image = Image.open(image_path)

    # Preprocess the image using the feature extractor
    inputs = extractor(images=image, return_tensors="pt")

    # Forward pass through the model
    outputs = model(**inputs)

    # Extract the embeddings from the output
    logits = outputs.logits
    return logits.detach().cpu().numpy().tolist()  # Convert to list

# Path to the main folder containing subfolders of images
main_folder_path = r"C:\Users\dt\Desktop\Ayesha\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train-data"

# List to store the records for batch insertion
inserts = []

# Iterate over each label subfolder in the main folder
for label_folder in os.listdir(main_folder_path):
    label_folder_path = os.path.join(main_folder_path, label_folder)
    
    # Check if it's a directory
    if os.path.isdir(label_folder_path):
        # Iterate over each image in the subfolder
        for image_name in os.listdir(label_folder_path):
            image_path = os.path.join(label_folder_path, image_name)
            
            # Check if it's a file
            if os.path.isfile(image_path):
                # Process the image and get the embedding
                embedding = process_image(image_path)
                
                # Create a record and append to the list
                inserts.append({
                    "image_embedding": embedding,
                    "label": label_folder,  # The folder name is used as the label
                })

# Now you would insert the data into the database
# For demonstration, we will just print the first few records
print(f"Inserts Preview: {inserts[:5]}")


c:\Users\dt\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\convnext\feature_extraction_convnext.py:28: FutureWarning: The class ConvNextFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ConvNextImageProcessor instead.
  warnings.warn(


Inserts Preview: [{'image_embedding': [[-9.559257507324219, -7.628460884094238, -12.010871887207031, -9.583788871765137, -10.490311622619629, -7.412520408630371, -10.010519981384277, -9.173770904541016, -8.42684555053711, -9.01956558227539, -5.568480968475342, -6.400491237640381, -7.399068832397461, -7.729429244995117, -6.849587917327881, -6.5220627784729, -5.620372772216797, -8.185742378234863, -8.289750099182129, -8.164188385009766, -5.074090003967285, -6.521359443664551, -8.00278091430664, -7.901402950286865, -9.533381462097168, -8.075019836425781, -5.661808967590332, -7.306702613830566, -7.815074920654297, -6.2730207443237305, -8.274026870727539, -5.156452178955078, -5.255512714385986, -7.26437520980835, -7.9994659423828125, -5.872614860534668, -6.197338104248047, -7.379458427429199, -3.6229166984558105, -6.882864952087402, -4.125380039215088, -7.073056221008301, -6.071549415588379, -5.714239597320557, -6.415278434753418, -7.673407077789307, -6.0049543380737305, -3.685746192932129,

In [73]:
print(f"Inserts Preview: {inserts[:1]}")

Inserts Preview: [{'image_embedding': [[-9.559257507324219, -7.628460884094238, -12.010871887207031, -9.583788871765137, -10.490311622619629, -7.412520408630371, -10.010519981384277, -9.173770904541016, -8.42684555053711, -9.01956558227539, -5.568480968475342, -6.400491237640381, -7.399068832397461, -7.729429244995117, -6.849587917327881, -6.5220627784729, -5.620372772216797, -8.185742378234863, -8.289750099182129, -8.164188385009766, -5.074090003967285, -6.521359443664551, -8.00278091430664, -7.901402950286865, -9.533381462097168, -8.075019836425781, -5.661808967590332, -7.306702613830566, -7.815074920654297, -6.2730207443237305, -8.274026870727539, -5.156452178955078, -5.255512714385986, -7.26437520980835, -7.9994659423828125, -5.872614860534668, -6.197338104248047, -7.379458427429199, -3.6229166984558105, -6.882864952087402, -4.125380039215088, -7.073056221008301, -6.071549415588379, -5.714239597320557, -6.415278434753418, -7.673407077789307, -6.0049543380737305, -3.685746192932129,

In [82]:
new_format_inserts = []

for item in inserts:
    new_item = {
        "label": item["label"],
        "image_embedding": [str(embedding) for embedding in item["image_embedding"][0]]
    }
    new_format_inserts.append(new_item)

# Now, new_format_inserts contains the data in the desired JSON format
print(new_format_inserts[0])
print(type(new_format_inserts))


{'label': 'Apple___Apple_scab', 'image_embedding': ['-9.559257507324219', '-7.628460884094238', '-12.010871887207031', '-9.583788871765137', '-10.490311622619629', '-7.412520408630371', '-10.010519981384277', '-9.173770904541016', '-8.42684555053711', '-9.01956558227539', '-5.568480968475342', '-6.400491237640381', '-7.399068832397461', '-7.729429244995117', '-6.849587917327881', '-6.5220627784729', '-5.620372772216797', '-8.185742378234863', '-8.289750099182129', '-8.164188385009766', '-5.074090003967285', '-6.521359443664551', '-8.00278091430664', '-7.901402950286865', '-9.533381462097168', '-8.075019836425781', '-5.661808967590332', '-7.306702613830566', '-7.815074920654297', '-6.2730207443237305', '-8.274026870727539', '-5.156452178955078', '-5.255512714385986', '-7.26437520980835', '-7.9994659423828125', '-5.872614860534668', '-6.197338104248047', '-7.379458427429199', '-3.6229166984558105', '-6.882864952087402', '-4.125380039215088', '-7.073056221008301', '-6.071549415588379', '-

In [83]:
import json


output_file_path = "output.json"

# Write the list to a JSON file
with open(output_file_path, "w") as json_file:
    json.dump(new_format_inserts, json_file)

print(f"Data written to {output_file_path}")


Data written to output.json


In [84]:
# Path to your JSON file
file_path = 'output.json'

# Load the data from the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# List to store the records for batch insertion
inserts = []

# Loop through each entry in your JSON data and prepare it for insertion
for item in data:
    label = item['label']
    # Convert string embeddings to a list of floats as required by Milvus
    embedding = [float(e) for e in item['image_embedding']]
    
    # Append the record to the list
    inserts.append({
        "image_embedding": embedding,
        "label": label,
    })

# Now insert the data into the collection
mr = collection.insert(inserts)

# You might want to print the insert result or handle it as needed
print(f"Insert Result: {mr}")

Insert Result: (insert count: 1900, delete count: 0, upsert count: 0, timestamp: 446245277424156676, success count: 1900, err count: 0)


In [85]:
# Convert image to mebeddings for vector search
image = Image.open("TomatoEarlyBlight4.JPG")

# Preprocess the image using the feature extractor
inputs = extractor(images=image, return_tensors="pt")

# Forward pass through the model
outputs = model(**inputs)

# Extract the embeddings from the output
print(outputs)

# Get the logits tensor from the model's outputs
logits = outputs.logits

ImageClassifierOutputWithNoAttention(loss=None, logits=tensor([[ -8.6919,  -8.4906, -11.2390,  -9.1138,  -8.8709,  -7.7781,  -8.9213,
          -9.6263,  -9.3359, -10.6544,  -9.3050,  -9.1450,  -9.5760,  -8.6133,
          -9.1466,  -8.2922,  -8.6782,  -9.3074,  -9.6370,  -9.2344,  -9.6399,
         -10.1245,  -9.6055, -10.1782,  -9.7773,  -9.5637,  -7.6098,  -7.8792,
          -8.6053,  -7.3681,  -9.0768,  -5.5125,  -5.7418,  -6.6531,  -6.3377,
          -6.9805,  -7.0798,  -7.4406,  -5.3702,  -7.6144,  -5.7806,  -7.7774,
          -7.6612,  -7.1923,  -7.4281,  -6.4124,  -6.7118,  -2.2905,  -9.8899,
          -8.6773,  -7.9113,  -7.3163,  -6.8038,  -8.9974,  -8.4602,  -7.4224,
          -9.3717,  -8.7127,  -7.8232,  -7.1563,  -7.9248,  -9.4673,  -8.1781,
          -8.1885,  -6.2146,  -8.9751,  -6.3427,  -6.7189,  -8.2657,  -8.4791,
          -6.9245,  -7.3803,  -8.5639,  -7.7375,  -8.4555,  -8.6601,  -9.1537,
          -9.1187,  -7.5646,  -7.8286,  -8.4616,  -8.4656,  -8.4115,  -7.906

In [86]:
# Convert PyTorch tensor to a list of lists
vectors_to_search = logits.detach().cpu().tolist()

# If logits is a 2D tensor with a single data point, it will be a nested list after tolist(),
# but Milvus expects a flat list for a single vector, so take the first element.
if len(vectors_to_search) == 1:
    vectors_to_search = vectors_to_search[0]

# Now perform the search with the vector
res = client.search(
    collection_name=COLLECTION_NAME,
    data=[vectors_to_search],  # Wrap the flat list in another list to make it a list of lists
    output_fields=["label"],
    limit=5
)

print(res)


[[{'id': 446011775355547475, 'distance': 0.99614018201828, 'entity': {'label': 'Tomato___Bacterial_spot'}}, {'id': 446011775355547803, 'distance': 0.9958937764167786, 'entity': {'label': 'Tomato___Spider_mites Two-spotted_spider_mite'}}, {'id': 446011775355547664, 'distance': 0.9957588911056519, 'entity': {'label': 'Tomato___Leaf_Mold'}}, {'id': 446011775355547492, 'distance': 0.9956377148628235, 'entity': {'label': 'Tomato___Bacterial_spot'}}, {'id': 446011775355547481, 'distance': 0.9955969452857971, 'entity': {'label': 'Tomato___Bacterial_spot'}}]]
